In [39]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
from CCLE_postp_function import CreateDatasetWithNewCellLines
sys.path.insert(0, '../JKBio/')
import datanalytics as da 
import TerraFunction as terra
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
namespace="broad-genomics-delivery"
workspace="Getz_IBM_CellLines_Exomes"
refnamespace="broad-firecloud-ccle"
refworkspace="DepMap_Mutation_Calling_CGA_pipeline"
source="ibm"
sample_set_id = "19Q3interim"

In [34]:
wm = dm.WorkspaceManager(namespace, workspace)
refwm = dm.WorkspaceManager(refnamespace, refworkspace)

In [49]:
a = ["ibm_ACH-001518",
"ibm_ACH-002019",
"ibm_ACH-002027",
"ibm_ACH-002069",
"ibm_ACH-001991",
"ibm_ACH-001719",
"ibm_ACH-001616",
"ibm_ACH-001623",
"ibm_ACH-002022",
"ibm_ACH-002042",
"ibm_ACH-002023",
"ibm_ACH-001687",
"ibm_ACH-001698"]
refwm.update_sample_set(sample_set_id=sample_set_id, sample_ids=a)

Sample set "19Q3interim" (13 samples) successfully updated.


In [47]:
CreateDatasetWithNewCellLines(wm,refwm, source, sample_set_id)

/home/jeremie/ccle_processing/CCLE_postp_function.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  samples = wfrom.get_samples()


uploading new samples
Successfully imported 1576 participants.
Updating many hound records. Switching to batch updates
Hound executing batch upload of 2 records
Successfully imported 1801 samples.
Updating many hound records. Switching to batch updates
Hound executing batch upload of 655566 records
creating a sample set
Successfully imported 1 sample sets:
  * 19Q3interim (1 samples)


# Run the pipeline

In [44]:
CGAmutations = refwm.get_config("ICE_CGA_Production_Analysis_Pipeline_Cell_Lines")
CGAmutations

{'deleted': False,
 'inputs': {'CGA_Production_Analysis_Workflow.CrossCheckLaneFingerprints_Task.HaplotypeDBForCrossCheck': 'workspace.HAP_DB_FOR_CC',
  'CGA_Production_Analysis_Workflow.gatk_acnv_only.one_thousand_genomes_common_variants_minor_allele_freq_five': 'workspace.COMMON_VARIANTS_1000_GENOMES',
  'CGA_Production_Analysis_Workflow.VEP_Task.GNOMAD_FILE_IDX': 'workspace.GNOMAD_IDX',
  'CGA_Production_Analysis_Workflow.lego_plotter_task.mut_categs': 'workspace.MUT_CATEGORIES',
  'CGA_Production_Analysis_Workflow.MutectFC_Task.mutectIntervals': 'workspace.MUTECT_FC_INTERVALS',
  'CGA_Production_Analysis_Workflow.hasPicardMetrics_normal': '',
  'CGA_Production_Analysis_Workflow.DB_SNP_VCF_IDX': 'workspace.DB_SNP_VCF_IDX',
  'CGA_Production_Analysis_Workflow.iceMAFPonFilter.PONFile': 'workspace.ICE_TOKEN_PON',
  'CGA_Production_Analysis_Workflow.ctrlName': '"CCLF_RCRF1009-Normal-SM-F3R8L"',
  'CGA_Production_Analysis_Workflow.targetIntervals': 'workspace.ILLUMINA_TARGET_INTERVALS',


In [ ]:
refwm.update_config(CGAmutations)
submission_id = refwm.create_submission(CGAmutations['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
wait_for_submission(refwm, submission_id1)

In [ ]:
filtercommonvar = refwm.get_config("common_variant_filter")
filtercommonvar

In [ ]:
refwm.update_config(filtercommonvar)
submission_id = refwm.create_submission(filtercommonvar['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
wait_for_submission(refwm, submission_id1)

In [ ]:
filtermaf = refwm.get_config("filterMAF_on_CGA_pipeline")
filtermaf

In [ ]:
refwm.update_config(filtermaf)
submission_id = refwm.create_submission(filtermaf['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
wait_for_submission(refwm, submission_id1)

In [ ]:
aggregate = refwm.get_config("aggregateMAFs_selectFields")
aggregate

In [ ]:
refwm.update_config(aggregate)
submission_id = refwm.create_submission(aggregate['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
wait_for_submission(refwm, submission_id1)

In [ ]:
%load_ext rpy2.ipython
release = sample_set_id

In [ ]:
%%R
release <- '19Q3interim'

In [ ]:
%%R
source('../JKBio/gkugener/RScripts/load_libraries_and_annotations.R')
load('../JKBio/gkugener/DM_OMICS/Annotations.RData') 
# There are some cell lines the celllinemapr does not know how to map so we need to load this data object for now (from old datasets)



In [ ]:
%%R
source('CCLE_postp_function.R')

In [ ]:
res = refwm.get_sample_sets().loc[samplesetname]
res = res['']
! gsutil cp $res "temp/mutation.mergedmaf.txt"


In [ ]:
%%R
li

In [ ]:
from taigapy import TaigaClient

tc = TaigaClient()
tc.update_dataset(dataset_permaname="depmap-fusions-7990",
                 upload_file_path_dict={'temp/fusions.'+release+'.filtered.tsv': 'TableTSV',
                                        'temp/fusions.'+release+'.unfiltered.tsv': 'TableTSV'},
                 dataset_description="Updating to "+release,
                 force_remove=True)